# Feature Store Quick Start

This notebook gives you a quick overview of how you can intergrate the feature store service on Hopsworks into your machine learning pipeline. We'll go over four steps:

1. Generate some sample data (rather than reading data from disk just to make this notebook stand-alone)
2. Do some feature engineering on the data
3. **Save the engineered features to the feature store**
4. **Select a group of the features from the feature store and create a managed training dataset of tf records in the feature store**
5. Train a model on the training dataset

## Imports

We'll use numpy and pandas for data generation, pyspark for feature engineering, tensorflow and keras for model training, and the hops `featurestore` library for interacting with the feature store.

In [24]:
import numpy as np
import random
import pandas as pd
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql import Row
from hops import featurestore
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Generate Sample Data

Lets generate two sample datasets:

1. `houses_for_sale_data`:

```bash
+-------+--------+------------------+------------------+------------------+
|area_id|house_id|       house_worth|         house_age|        house_size|
+-------+--------+------------------+------------------+------------------+
|      1|       0| 11678.15482418699|133.88670106643886|366.80067322738535|
|      1|       1| 2290.436167500643|15994.969706808222|195.84014889823976|
|      1|       2| 8380.774578431328|1994.8576926471007|1544.5164614303735|
|      1|       3|11641.224696102923|23104.501275562343|1673.7222604337876|
|      1|       4| 5382.089422436954| 13903.43637058141| 274.2912104765028|
+-------+--------+------------------+------------------+------------------+

 |-- area_id: long (nullable = true)
 |-- house_id: long (nullable = true)
 |-- house_worth: double (nullable = true)
 |-- house_age: double (nullable = true)
 |-- house_size: double (nullable = true)
```
2. `houses_sold_data``
```bash
+-------+-----------------+-----------------+------------------+
|area_id|house_purchase_id|number_of_bidders|   sold_for_amount|
+-------+-----------------+-----------------+------------------+
|      1|                0|                0| 70073.06059070028|
|      1|                1|               15| 146.9198329740602|
|      1|                2|                6|  594.802165433149|
|      1|                3|               10| 77187.84123130841|
|      1|                4|                1|110627.48922722359|
+-------+-----------------+-----------------+------------------+

 |-- area_id: long (nullable = true)
 |-- house_purchase_id: long (nullable = true)
 |-- number_of_bidders: long (nullable = true)
 |-- sold_for_amount: double (nullable = true)
```

We'll use this data for predicting what a house is sold for based on features about the **area** where the house is.

### Generation of `houses_for_sale_data`

In [4]:
area_ids = list(range(1,51))
house_sizes = []
house_worths = []
house_ages = []
house_area_ids = []
for i in area_ids:
    for j in list(range(1,100)):
        house_sizes.append(abs(np.random.normal()*1000)/i)
        house_worths.append(abs(np.random.normal()*10000)/i)
        house_ages.append(abs(np.random.normal()*10000)/i)
        house_area_ids.append(i)
house_ids = list(range(len(house_area_ids)))
houses_for_sale_data  = pd.DataFrame({
        'area_id':house_area_ids,
        'house_id':house_ids,
        'house_worth': house_worths,
        'house_age': house_ages,
        'house_size': house_sizes
    })
houses_for_sale_data_spark_df = sqlContext.createDataFrame(houses_for_sale_data)

In [6]:
houses_for_sale_data_spark_df.show(5)

+-------+--------+------------------+------------------+------------------+
|area_id|house_id|       house_worth|         house_age|        house_size|
+-------+--------+------------------+------------------+------------------+
|      1|       0| 531.9876527313653| 8114.466859391631|155.25702875144782|
|      1|       1|15344.430783478807|3147.5665452473127| 83.71893254128386|
|      1|       2|18357.569890383398|1199.1734241476324| 854.9770429830093|
|      1|       3| 2807.313400895764| 540.1950456572356|31.025385992686427|
|      1|       4|2160.0170197342927| 2563.566293972737|271.35295036614554|
+-------+--------+------------------+------------------+------------------+
only showing top 5 rows

In [7]:
houses_for_sale_data_spark_df.printSchema()

root
 |-- area_id: long (nullable = true)
 |-- house_id: long (nullable = true)
 |-- house_worth: double (nullable = true)
 |-- house_age: double (nullable = true)
 |-- house_size: double (nullable = true)

### Generation of `houses_sold_data`

In [8]:
house_purchased_amounts = []
house_purchases_bidders = []
house_purchases_area_ids = []
for i in area_ids:
    for j in list(range(1,1000)):
        house_purchased_amounts.append(abs(np.random.exponential()*100000)/i)
        house_purchases_bidders.append(int(abs(np.random.exponential()*10)/i))
        house_purchases_area_ids.append(i)
house_purchase_ids = list(range(len(house_purchases_bidders)))
houses_sold_data  = pd.DataFrame({
        'area_id':house_purchases_area_ids,
        'house_purchase_id':house_purchase_ids,
        'number_of_bidders': house_purchases_bidders,
        'sold_for_amount': house_purchased_amounts
    })
houses_sold_data_spark_df = sqlContext.createDataFrame(houses_sold_data)

In [9]:
houses_sold_data_spark_df.show(5)

+-------+-----------------+-----------------+------------------+
|area_id|house_purchase_id|number_of_bidders|   sold_for_amount|
+-------+-----------------+-----------------+------------------+
|      1|                0|                3| 77660.62755542179|
|      1|                1|                6| 22192.75497087212|
|      1|                2|                4| 12804.50521418885|
|      1|                3|                6|204152.64327486677|
|      1|                4|               21|59384.149525606976|
+-------+-----------------+-----------------+------------------+
only showing top 5 rows

In [10]:
houses_sold_data_spark_df.printSchema()

root
 |-- area_id: long (nullable = true)
 |-- house_purchase_id: long (nullable = true)
 |-- number_of_bidders: long (nullable = true)
 |-- sold_for_amount: double (nullable = true)

## Feature Engineering

Lets generate some aggregate features such as sum and averages from our datasets. 

1. `houses_for_sale_features`:

```bash
 |-- area_id: long (nullable = true)
 |-- avg_house_age: double (nullable = true)
 |-- avg_house_size: double (nullable = true)
 |-- avg_house_worth: double (nullable = true)
 |-- sum_house_age: double (nullable = true)
 |-- sum_house_size: double (nullable = true)
 |-- sum_house_worth: double (nullable = true)
```

2. `houses_sold_features`

```bash
 |-- area_id: long (nullable = true)
 |-- avg_num_bidders: double (nullable = true)
 |-- avg_sold_for: double (nullable = true)
 |-- sum_number_of_bidders: long (nullable = true)
 |-- sum_sold_for_amount: double (nullable = true)
```

### Generate Features From `houses_for_sale_data`

In [11]:
sum_houses_for_sale_df = houses_for_sale_data_spark_df.groupBy("area_id").sum()
count_houses_for_sale_df = houses_for_sale_data_spark_df.groupBy("area_id").count()
sum_count_houses_for_sale_df = sum_houses_for_sale_df.join(count_houses_for_sale_df, "area_id")
sum_count_houses_for_sale_df = sum_count_houses_for_sale_df \
    .withColumnRenamed("sum(house_age)", "sum_house_age") \
    .withColumnRenamed("sum(house_worth)", "sum_house_worth") \
    .withColumnRenamed("sum(house_size)", "sum_house_size") \
    .withColumnRenamed("count", "num_rows")
def compute_average_features_house_for_sale(row):
    avg_house_worth = row.sum_house_worth/float(row.num_rows)
    avg_house_size = row.sum_house_size/float(row.num_rows)
    avg_house_age = row.sum_house_age/float(row.num_rows)
    return Row(
        sum_house_worth=row.sum_house_worth, 
        sum_house_age=row.sum_house_age,
        sum_house_size=row.sum_house_size,
        area_id = row.area_id,
        avg_house_worth = avg_house_worth,
        avg_house_size = avg_house_size,
        avg_house_age = avg_house_age
       )
houses_for_sale_features_df = sum_count_houses_for_sale_df.rdd.map(
    lambda row: compute_average_features_house_for_sale(row)
).toDF()

In [13]:
houses_for_sale_features_df.printSchema()

root
 |-- area_id: long (nullable = true)
 |-- avg_house_age: double (nullable = true)
 |-- avg_house_size: double (nullable = true)
 |-- avg_house_worth: double (nullable = true)
 |-- sum_house_age: double (nullable = true)
 |-- sum_house_size: double (nullable = true)
 |-- sum_house_worth: double (nullable = true)

### Generate Features from `houses_sold_data`

In [14]:
sum_houses_sold_df = houses_sold_data_spark_df.groupBy("area_id").sum()
count_houses_sold_df = houses_sold_data_spark_df.groupBy("area_id").count()
sum_count_houses_sold_df = sum_houses_sold_df.join(count_houses_sold_df, "area_id")
sum_count_houses_sold_df = sum_count_houses_sold_df \
    .withColumnRenamed("sum(number_of_bidders)", "sum_number_of_bidders") \
    .withColumnRenamed("sum(sold_for_amount)", "sum_sold_for_amount") \
    .withColumnRenamed("count", "num_rows")
def compute_average_features_houses_sold(row):
    avg_num_bidders = row.sum_number_of_bidders/float(row.num_rows)
    avg_sold_for = row.sum_sold_for_amount/float(row.num_rows)
    return Row(
        sum_number_of_bidders=row.sum_number_of_bidders, 
        sum_sold_for_amount=row.sum_sold_for_amount,
        area_id = row.area_id,
        avg_num_bidders = avg_num_bidders,
        avg_sold_for = avg_sold_for
       )
houses_sold_features_df = sum_count_houses_sold_df.rdd.map(
    lambda row: compute_average_features_houses_sold(row)
).toDF()

In [15]:
houses_sold_features_df.printSchema()

root
 |-- area_id: long (nullable = true)
 |-- avg_num_bidders: double (nullable = true)
 |-- avg_sold_for: double (nullable = true)
 |-- sum_number_of_bidders: long (nullable = true)
 |-- sum_sold_for_amount: double (nullable = true)

## Save Features to the Feature Store

The Featue store has an abstraction of a **feature group** which is a set of features that naturally belong together that typically are computed using the same feature engineering job and the same raw dataset. 

Lets create two feature groups:

1. `houses_for_sale_featuregroup`

2. `houses_sold_featuregroup`

In [16]:
featurestore.create_featuregroup(
    houses_for_sale_features_df,
    "houses_for_sale_featuregroup",
    description="aggregate features of houses for sale per area",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False
)

Running sql: use fs_demo_featurestore
Running sql: use fs_demo_featurestore
Running sql: SELECT * FROM houses_for_sale_featuregroup_1

In [17]:
featurestore.create_featuregroup(
    houses_sold_features_df,
    "houses_sold_featuregroup",
    description="aggregate features of sold houses per area",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False
)

Running sql: use fs_demo_featurestore
Running sql: use fs_demo_featurestore
Running sql: SELECT * FROM houses_sold_featuregroup_1

## Create a Training Dataset

The feature store has an abstraction of a **training dataset**, which is a dataset with a set of features (potentially from many different feature groups) and labels (in case of supervised learning). 

Let's create a training dataset called *predict_house_sold_for_dataset* using the following features:

- avg_house_age
- avg_house_size
- avg_house_worth
- avg_num_bidders

and the target variable is:

- avg_sold_for

In [22]:
features_df = featurestore.get_features(["avg_house_age", "avg_house_size", 
                                         "avg_house_worth", "avg_num_bidders", 
                                         "avg_sold_for"])
featurestore.create_training_dataset(
    features_df, "predict_house_sold_for_dataset",
    data_format="tfrecords",
    descriptive_statistics=False,
    feature_correlation=False,
    feature_histograms=False,
    cluster_analysis=False
)

Running sql: use fs_demo_featurestore
Running sql: SELECT avg_sold_for, avg_house_age, avg_house_size, avg_num_bidders, avg_house_worth FROM houses_sold_featuregroup_1 JOIN houses_for_sale_featuregroup_1 ON houses_sold_featuregroup_1.`area_id`=houses_for_sale_featuregroup_1.`area_id`

## Use the Training Dataset to Train a Model

When creating training datasets through the feature store API, the training dataset becomes *managed* by Hopsworks, meaning that it will get automatic versioning, documentation, API support, and analysis. 

Let's create a simple neural network and train it for the regression task of predicting the target variable `avg_sold_for`.

In [27]:
NUM_EPOCHS = 30
BATCH_SIZE = 10
LEARNING_RATE = 0.001
SHUFFLE_BUFFER_SIZE = 10000
dataset_dir = featurestore.get_training_dataset_path("predict_house_sold_for_dataset")
input_files = tf.gfile.Glob(dataset_dir + "/part-r-*")
dataset = tf.data.TFRecordDataset(input_files)
tf_record_schema = featurestore.get_training_dataset_tf_record_schema("predict_house_sold_for_dataset")
feature_names = ["avg_house_age", "avg_house_size", "avg_house_worth", "avg_num_bidders"]
label_name = "avg_sold_for"
def decode(example_proto):
    example = tf.parse_single_example(example_proto, tf_record_schema)
    x = []
    for feature_name in feature_names:
        x.append(example[feature_name])
    y = tf.cast(example[label_name], tf.float32)
    return x,y
dataset = dataset.map(decode).shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).repeat(NUM_EPOCHS)
model = tf.keras.Sequential([
layers.Dense(64, activation='relu'),
layers.Dense(64, activation='relu'),
layers.Dense(1)])
model.compile(optimizer=tf.train.AdamOptimizer(LEARNING_RATE), loss='mse')
history = model.fit(dataset, epochs=NUM_EPOCHS, steps_per_epoch = 5)

Epoch 1/30
5/5 [==============================] - 1s 269ms/step - loss: 318853652.0000
Epoch 2/30
5/5 [==============================] - 1s 165ms/step - loss: 316778487.6000
Epoch 3/30
5/5 [==============================] - 1s 168ms/step - loss: 315049218.4000
Epoch 4/30
5/5 [==============================] - 1s 141ms/step - loss: 313515144.8000
Epoch 5/30
5/5 [==============================] - 1s 157ms/step - loss: 311750453.6000
Epoch 6/30
5/5 [==============================] - 1s 171ms/step - loss: 309745111.2000
Epoch 7/30
5/5 [==============================] - 1s 146ms/step - loss: 307866987.2000
Epoch 8/30
5/5 [==============================] - 1s 163ms/step - loss: 307098876.8000
Epoch 9/30
5/5 [==============================] - 1s 173ms/step - loss: 304209784.8000
Epoch 10/30
5/5 [==============================] - 1s 279ms/step - loss: 300723418.8000
Epoch 11/30
5/5 [==============================] - 1s 133ms/step - loss: 299241663.6000
Epoch 12/30
5/5 [========================